In [ ]:
CREATE TABLE emp_tbl (id datetime, empid int);

INSERT INTO emp_tbl VALUES ('2024-01-13 09:25:00', 10),('2024-01-13 19:35:00', 10),('2024-01-16 09:10:00', 10),
('2024-01-16 18:10:00', 10),('2024-02-11 09:07:00', 10),('2024-02-11 19:20:00', 10),('2024-02-17 08:40:00', 17),
('2024-02-17 18:04:00', 17),('2024-03-23 09:20:00', 10),('2024-03-23 18:30:00', 10);

In [ ]:
select empid,day(id) as day from emp_tbl where dayname(id) in ('Sat','Sun') group by empid,day(id);

In [ ]:
create table tbl_maxval (col1 varchar(50), col2 int, col3 int);
insert into tbl_maxval values ('a',10,20),('b',50,30);

In [ ]:
select col1,least(col2,col3) from tbl_maxval;

In [ ]:
create table emp(empid int, empname varchar(50), salary int, deptid int);
insert into emp values (1,'Nikitha',45000,206),(2,'Ashish',42000,207),(3,'David',40000,206),(4,'Ram',50000,207),(5,'John',35000,208),(6,'Mark',50000,207),(7,'Aravind',39000,208);


create table dept (deptid int, deptname varchar(50));
insert into dept values (206,'HR'),(207,'IT'),(208,'Finance');

In [ ]:
select * from emp;

In [ ]:
with cte as(
select *,dense_rank() over(partition by e.deptid order by e.salary desc) as rank from emp e inner join dept d on e.deptid=d.deptid ),cte2 as(
select * from cte where rank=1
)
select deptname,listagg(empname,',') as maxsal from cte2 group by deptname;


In [ ]:
select * from dept;

In [ ]:
create table empdept_tbl (eid int, dept varchar(50),scores float);
insert into empdept_tbl values (1, 'd1', 1.0),(2, 'd1', 5.28),(3, 'd1', 4.0),(4,'d2', 8.0),(5, 'd1', 2.5),(6, 'd2', 7.0),(7, 'd3', 9.0),(8, 'd4', 10.2);

In [ ]:
select *,max(scores) over(partition by dept order by dept) as maxscore from empdept_tbl;

In [ ]:
with cte as(
select dept,max(scores) as maxscore from empdept_tbl e group by dept order by dept asc)
select e.*,c.maxscore as maxempscore
from cte c inner join empdept_tbl e on c.dept=e.dept

In [ ]:
WITH MaxScores AS (
  SELECT dept, MAX(scores) AS max_score
  FROM empdept_tbl
  GROUP BY dept
)

SELECT 
  e.eid, 
  e.dept, 
  e.scores, 
  m.max_score AS max_score_for_dept
FROM 
  empdept_tbl e
JOIN 
  MaxScores m ON e.dept = m.dept;

In [ ]:
create table prd_tbl (dt date, brand varchar(50), model varchar(50),
production_cost int);

insert into prd_tbl values ('2023-12-01', 'A', 'A1', 1000), ('2023-12-01', 'A', 'A2', 1300),('2023-12-01', 'B', 'B1', 800), ('2023-12-02', 'A', 'A1', 1800),('2023-12-02', 'B', 'B1', 900), ('2023-12-10', 'A', 'A1', 1400),('2023-12-10', 'A', 'A1', 1200), ('2023-12-10', 'C', 'C1', 2500);

In [ ]:
select * from prd_tbl;

In [ ]:
with cte as(
select dt,brand,sum(production_cost) as totalproduction from prd_tbl group by dt,brand)
select p.*,c.totalproduction as totalproduction from cte c inner join prd_tbl p on c.brand=p.brand and c.dt=p.dt;

In [ ]:
select *,sum(production_cost) over(partition by dt,brand order by dt) as agg_cost from prd_tbl;

In [ ]:
CREATE TABLE exchange_rates (
    currency_code VARCHAR(3),
    date DATE,
    currency_exchange_rate DECIMAL(10, 2)
);

INSERT INTO exchange_rates (currency_code, date, currency_exchange_rate) VALUES
('USD', '2024-06-01', 1.20),
('USD', '2024-06-02', 1.21),
('USD', '2024-06-03', 1.22),
('USD', '2024-06-04', 1.23),
('USD', '2024-07-01', 1.25),
('USD', '2024-07-02', 1.26),
('USD', '2024-07-03', 1.27),
('EUR', '2024-06-01', 1.40),
('EUR', '2024-06-02', 1.41),
('EUR', '2024-06-03', 1.42),
('EUR', '2024-06-04', 1.43),
('EUR', '2024-07-01', 1.45),
('EUR', '2024-07-02', 1.46),
('EUR', '2024-07-03', 1.47);

In [ ]:
select * from exchange_rates;

In [ ]:
with cte as (
select *,concat(currency_code,'-',year(date),'-',month(date)) as currency_codetype_year_month ,row_number() over(partition by currency_code,year(date),month(date) order by date) as rownum_asc,row_number() over(partition by currency_code,year(date),month(date) order by date desc) as rownum_desc from exchange_rates )
select currency_codetype_year_month,max((case when rownum_asc=1 then currency_exchange_rate end)) as currency_exchange_beginning_of_month,
max((case when rownum_desc=1 then currency_exchange_rate end)) as currency_exchange_end_of_month
from cte group by currency_codetype_year_month;

In [ ]:
CREATE TABLE department (deptid INT, deptname VARCHAR(50));
INSERT INTO department VALUES (101, 'HR'), (102, 'Finance'), (103, 'Marketing');

CREATE TABLE employee (empid INT, salary INT, deptid INT);
INSERT INTO employee VALUES (1, 70000, 101),(2, 50000, 101),(3, 60000, 101),(4, 65000, 102),(5, 65000, 102),(6, 55000, 102),(7, 60000, 103),(8, 70000, 103),(9, 80000, 103);


In [ ]:
select * from department;

In [ ]:
select * from employee;

In [ ]:
select e.empid,e.salary,e.deptid,d.deptname,dense_rank() over(partition by deptname order by salary desc) as rank from employee e inner join department d on e.deptid=d.deptid

In [ ]:
with cte as(
select e.empid,e.salary,e.deptid,d.deptname,dense_rank() over(partition by deptname order by salary desc) as rank from employee e inner join department d on e.deptid=d.deptid)
select * from cte where rank=2;


In [ ]:
CREATE TABLE employees_tbl (eid INT, ename VARCHAR(50), gender VARCHAR(10));

INSERT INTO employees_tbl VALUES (1, 'John Doe', 'Male'),(2, 'Jane Smith', 'Female'),
(3, 'Michael Johnson', 'Male'),(4, 'Emily Davis', 'Female'),(5, 'Robert Brown', 'Male'),
(6, 'Sophia Wilson', 'Female'),(7, 'David Lee', 'Male'),(8, 'Emma White', 'Female'),
(9, 'James Taylor', 'Male'),(10, 'William Clark', 'Male');


In [ ]:
select * from employees_tbl;

In [ ]:
select current_role()

In [ ]:
use accountadmin;
create or replace masking policy mymaskingpolicy as (val string) returns string-> case when current_role() in ('ACCOUNTADMIN') then val else repeat('*',len(val)) end;

In [ ]:
with cte as (
select count(gender) as total_person from employees_tbl),cte2 as(
select count(*) as male_count from employees_tbl where gender='Male'
),cte3 as(

select count(*) as female_count from employees_tbl where gender='Female' 
)
select ((cte2.male_count/cte.total_person)*100) as male_per,((cte3.female_count/cte.total_person)*100) as female_per from cte,cte2,cte3 ;


In [ ]:
select count(*) as male_count from employees_tbl where gender='Female' ;

In [ ]:
create table products (pid int, pname varchar(50), price int);
insert into products values (1, 'A', 1000),(2, 'B', 400),(3, 'C', 500);

create table transcations (pid int, sold_date DATE, qty int, amount int);
insert into transcations values (1, '2024-02-01', 2, 2000),(1, '2024-03-01', 4, 4000),
(1, '2024-03-15', 2, 2000),(3, '2024-04-24', 3, 1500),(3, '2024-05-16', 5, 2500);

2nd wednesday of current month

In [ ]:
select date_from_parts(year(current_date),month(current_date),01) as startdate

In [ ]:
with recursive cte as(
select date_from_parts(year(current_date),month(current_date),01) as startdate,
dayname(current_date) as dayname

union all
select dateadd(day,1,startdate) as startdate_f,
dayname(startdate_f) as dayname
from cte where startdate_f<='2024-11-30'

),cte2 as(
select startdate,dayname,row_number() over(order by dayname) as rn from cte where dayname='Wed')
select startdate from cte2 where rn=2;

In [ ]:

CREATE TABLE emps_tbl (emp_name VARCHAR(50), dept_id INT, salary INT);

INSERT INTO emps_tbl VALUES ('Siva', 1, 30000), ('Ravi', 2, 40000), ('Prasad', 1, 50000), ('Sai', 2, 20000), ('Anna', 2, 10000);

In [ ]:
select EMP_NAME from emps_tbl order by right("EMP_NAME",2);

In [ ]:
with cte as (
select emp_name,dept_id,salary,row_number() over(partition by dept_id order by salary) as rn_asc,row_number() over(partition by dept_id order by salary desc) as rn_desc from emps_tbl )
select dept_id,max(case when rn_asc=1 then emp_name end) lowestsal,max(case when rn_desc=1 then emp_name end )highestsal from cte group by dept_id order by dept_id asc;


In [ ]:
CREATE TABLE STUDENTS (id int, name string, marks INT);

INSERT INTO STUDENTS VALUES (1, 'Ashlay', 81), (2, 'Samantha', 75), (3, 'Julia', 76), (4, 'Belvet', 84);

In [ ]:
select * from STUDENTS

In [ ]:
select Name from STUDENTS where Marks>75 order by right(Name,3),ID asc

In [ ]:
CREATE TABLE transactions_1308 (transaction_id BIGINT, type VARCHAR(50), amount INT,transaction_date DATE);

-- Insert data into the table
INSERT INTO transactions_1308 VALUES (53151, 'deposit', 178, '2022-07-08'), (29776, 'withdrawal', 25, '2022-07-08'),(16461, 'withdrawal', 45, '2022-07-08'),(19153, 'deposit', 65, '2022-07-10'),(77134, 'deposit', 32, '2022-07-10');


In [ ]:
select * from transactions_1308;

In [ ]:
select *,sum(case when type='deposit' then amount else -amount end) over(order by transaction_date) as runningamount from transactions_1308

In [ ]:
with cte as(
select *,row_number() over(partition by transaction_date order by (select null)) as rn from transactions_1308
)
select *,sum(case when type= 'deposit' then amount else -amount end) over(order by transaction_date,rn) as runningamount from cte;

In [ ]:
CREATE TABLE sls_tbl (pid INT, sls_dt DATE, sls_amt INT );

-- Insert data into the table
INSERT INTO sls_tbl (pid, sls_dt, sls_amt)
VALUES (201, '2024-07-11', 140), (201, '2024-07-18', 160), (201, '2024-07-25', 150), (201, '2024-08-01', 180), (201, '2024-08-15', 170), (201, '2024-08-29', 130);


In [ ]:
select * from sls_tbl;

In [ ]:
select DATE_FROM_PARTS(2024,07,11)

In [ ]:
select SLS_DT from sls_tbl

In [ ]:
with recursive cte as(
select DATE_FROM_PARTS(2024,07,11) as startdate
union all
select dateadd(day,7,startdate) as date_r from cte
where date_r<='2024-08-29'

)select startdate as sls_date from cte WHERE startdate not in (select SLS_DT from sls_tbl ) ;

In [ ]:
--Create table syntax
CREATE TABLE testtbl (cola VARCHAR(10));

-- Insert data into the table
INSERT INTO testtbl (cola)
VALUES ('1,2'), ('3'), ('4');


In [ ]:
SELECT to_number(VALUE) as value
FROM testtbl,
     LATERAL FLATTEN(INPUT => SPLIT(cola, ','))

In [ ]:
-- Create table
CREATE TABLE user_activity (
    date DATE,
    user_id INT,
    activity VARCHAR(10)
);
 
-- Insert data
INSERT INTO user_activity (date, user_id, activity) VALUES
('2022-02-20', 1, 'abc'),
('2022-02-20', 2, 'xyz'),
('2022-02-22', 1, 'xyz'),
('2022-02-22', 3, 'klm'),
('2022-02-24', 1, 'abc'),
('2022-02-24', 2, 'abc'),
('2022-02-24', 3, 'abc');

In [ ]:
select * from user_activity;

In [ ]:
select *,row_number() over(partition by user_id order by (select null)) as rn from user_activity

In [ ]:
with cte as (
select *,row_number() over(partition by user_id order by (select null)) as rn from user_activity) select count(user_id) from cte group by rn;

In [ ]:
select count(user_id) from user_activity group by date

In [ ]:
with cte as(
select *,row_number() over(partition by user_id order by user_id) as rn from user_activity),cte2 as(
select count(*) from user_activity group by date

)

In [ ]:
CREATE TABLE MATCH(TEAM VARCHAR(20));
 
INSERT INTO MATCH VALUES ('INDIA'),('PAK'),('AUS'),('ENG');

In [ ]:
select m1.team as team_A,m2.team as team_B from MATCH m1 join MATCH m2 on m1.team<m2.team order by team_A,team_B

In [ ]:
select least(m1.team,m2.team) as team from MATCH m1 join MATCH m2 order by team 

In [ ]:
lateral(select team from match)

In [ ]:
select team from match , lateral(select team from match);

In [ ]:
select  date,count(distinct user_id)from user_activity

In [ ]:
select distinct(user_id) from  user_activity

In [ ]:
select date,listagg(distinct user_id,',') from  user_activity group by date

CREATE TABLE SEARCH_FREQUENCY(
  SEARCHES INT,
  NUM_USER INT
);
 
INSERT INTO SEARCH_FREQUENCY VALUES
(1,2),(4,1),(2,2),(3,3),(6,1),(5,3),(7,2);
 
Write a query to report the median of searches made by a user. Round the median to one decimal Point.
 
OUTPUT of this question:- Median  
                                           3.5

In [ ]:
CREATE TABLE SEARCH_FREQUENCY(
  SEARCHES INT,
  NUM_USER INT
);
 
INSERT INTO SEARCH_FREQUENCY VALUES
(1,2),(4,1),(2,2),(3,3),(6,1),(5,3),(7,2);


SELECT table1.value
  FROM TABLE(SPLIT_TO_TABLE('a.b', '.')) AS table1
  ORDER BY table1.value

In [ ]:
select split_to_table(listagg(repeat((to_char(searches)||','),num_user),''),',')::int as num from SEARCH_FREQUENCY

In [ ]:
SELECT 
    repeat(to_char(searches),num_user)
    FROM SEARCH_FREQUENCY 

In [ ]:
with recursive cte as(
select searches as s,num_user as nu,row_number() over(order by searches) as rn from SEARCH_FREQUENCY
union all
select repeat(to_char(s),nu) as repnum from cte where rn=count(*)

)select repnum from cte;

In [ ]:
select replace(REPLACE(listagg(repeat(to_char(searches),num_user),''),',')from SEARCH_FREQUENCY

In [ ]:
select count(searches) from SEARCH_FREQUENCY group by 

In [ ]:

CREATE TABLE events ( pid INT, year INT ) ;
-- Insert data into the table 
INSERT INTO events VALUES (1, 2019), (1, 2020), (1, 2021), (2, 2022), (2, 2021),(3, 2019), (3, 2021), (3, 2022);

In [ ]:
with cte as(
select *,lag(year,1,0) over(order by pid) as previousyear from events)
select year-previousyear